In [1]:
import numpy as np
from itertools import product
import cvxpy as cvx
import traceback
import itertools
from matplotlib import pyplot as plt
from scipy.stats import entropy

In [2]:
from Ensembles.Ensemble import Ensemble
from Ensembles.Cifar10.Accessor.readCifar10 import *

In [3]:
import import_ipynb
from spectralEM import spectralEM
from conflictGraph import conflictGraph
from fakeENS import fakeENS

importing Jupyter notebook from spectralEM.ipynb
importing Jupyter notebook from conflictGraph.ipynb
importing Jupyter notebook from fakeENS.ipynb


In [4]:
def kbits(n, k):
    result = []
    for bits in itertools.combinations(range(n), k):
        s = np.zeros(n)
        for bit in bits:
            s[bit] = 1
        result.append(s)
    return result

In [5]:
def str_arr(arr, mode = 'float'):
    if mode == 'int':
        return " ".join("%2d"%x for x in arr)
    else:
        return " ".join("%.3f"%x for x in arr)
def str_mat(mat):
    return "\n".join(str_arr(arr) for arr in mat) 

In [6]:
def averageAcc(p_true, C):
    return np.sum(C[i,i]*p for i, p in enumerate(p_true))    

In [7]:
def computeError(output_stream, true_label_stream):
    if len(output_stream) == 0:
        return 0
    est_label = [o[1] for o in output_stream]
    error = [(np.abs(x-y)>0.5) for x,y in zip(est_label, true_label_stream)]
    error_frac = (np.sum(error)/float(len(error)))
    return error_frac

In [8]:
def ConfError(C_real, num_data, p_true, num_classifier, C_est, ph_est_avg):
    errp = np.linalg.norm(p_true - ph_est_avg, 1)
    err = [np.linalg.norm(C_real[k]- C_est[k], 1) for k in range(num_classifier)]
    print '---------------------------'
    print '*****Param Update*****'
    print '---------------------------'
    print 'Num samples:%d'%num_data
    print 'True ph:%s'%str_arr(p_true)
    print 'Est ph:%s'%str_arr(ph_est_avg)
    print 'L1 error in class probability:%.3f'%errp
    print 'Sum of L1 error in conf matrices:%.3f'%np.sum(err)
    print '---------------------------'
    return errp, err

In [9]:
def simulator(data_set = 'Cifar10',
              num_class = 3, num_classifier = 5,  # Ensemble and Data
              load_ENS = False, fit_ENS = False, 
              load_params = True, save_params = False,# Ensemble
              k_max_size = 3, # allowed subset 
              arr_rate = 0.8, thres = 0.75, V = 1, # arrival and departure thres
              updateON = True, truePrior = False, # priors
              time_slots = 1000 # simulation length
             ):
    '''
    The function that simulates the system with the input parameters
    
    '''
    #---------------------------------
    #       ENS and Data
    #---------------------------------
    if load_ENS:
        ENS=Ensemble(data_set)
        try:
            # fit the ENS again
            if fit_ENS:
                ENS.get_train_data(); ENS.assign_members_train_data()
                ENS.get_test_data(); ENS.assign_members_test_data()
                ENS.create_classifiers(); ENS.fit_classifiers()
                ENS.save_classifiers()
            ENS.load_classifiers()
        except:
            print 'Error in loading classifiers'
            ENS.get_train_data(); ENS.assign_members_train_data()
            ENS.get_test_data(); ENS.assign_members_test_data()
            ENS.create_classifiers(); ENS.fit_classifiers()
            ENS.save_classifiers()
            ENS.load_classifiers()
            
        ENS.get_train_data(); ENS.assign_members_train_data()
        ENS.get_test_data(); ENS.assign_members_test_data()
        
        # info about classifiers
        name_classifier = ENS.import_name_classifier()
        num_classifier = ENS.import_num_classifier()
        num_class = ENS.import_num_class()
        
        # obtain the samples
        true_labels = ENS.import_labels()
        num_tot_samp_stream = len(true_labels)
        samples = np.arange(num_tot_samp_stream)
        
        # permute the samples and the labels
        perm_data = np.random.permutation(num_tot_samp_stream)
        stream_true_labels = true_labels[perm_data] # Begins from 0
        stream_sample = samples[perm_data] # Begins from 0
        
        # create arrival array and change time_slots if sample is less
        arrival_arr = np.random.poisson(arr_rate, time_slots)
        if np.sum(arrival_arr) > num_tot_samp_stream: 
            time_slots = np.argmax(np.cumsum(arrival_arr)> num_tot_samp_stream)-1
        num_tot_samp = np.sum(arrival_arr[:time_slots])
        
        print 'Number of total samples in stream:%d'%num_tot_samp_stream
        print 'Number of timeslots:%d'%time_slots
        print 'Number of arrivals:%d'%num_tot_samp
        print
        
        # obtain the parameters (w.r.t. test data)
        if load_params:
            trueConfMat = np.load('conf_mat.npy').item()
            p_true = np.load('p_true.npy')
        else:
            trueConfMat = ENS.get_conf_matrix()
            p_true = ENS.get_p_true()
    else:
        if load_params:
            conf_mat = np.load('conf_mat.npy').item() 
            p_true = np.load('p_true.npy')
            #print p_true
            #print conf_mat.items()
        else:
            # probability of the various classes
            p_const = 0.5
            p_true = np.random.rand(num_class); 
            p_true = (p_const+p_true)/ np.sum(p_const+p_true)
            conf_mat = None
        #-----------------------------------
        # synthetic dataset
        name_classifier = ['syn-'+str(i) for i in range(num_classifier)]
        ENS = fakeENS(num_classifier, num_class, confMat = conf_mat)
        trueConfMat = ENS.getConfMat()
        #------------------------------------
        # beginning of the simulation
        # can be restarted to rerun the simulation
        # rerunning following blocks(including this one)
        # do not change the capacity region
        arrival_arr = np.random.poisson(arr_rate, time_slots)
        num_tot_samp = np.sum(arrival_arr) 

        # generate the samples(both begins from 0)
        stream_sample = np.arange(num_tot_samp) 
        stream_true_labels = np.random.choice(num_class,
                                              num_tot_samp, p = p_true)
        
        # feed samples to the ensemble
        ENS.newSamples(stream_true_labels) 
        # re-randomize the output label mapping
        ENS.reshuffle() 
    #--------------------------------------------------------
    #.         Display Ensemble and Data
    #-------------------------------------------------------- 
    if save_params:
        np.save('conf_mat.npy', trueConfMat)
        np.save('p_true.npy', p_true)
    #-----------------------------------
    print "Number of class:%d"%num_class
    print "Number of classifiers:%d"%num_classifier
    print "Name of classifiers:",name_classifier
    print 'True label probability:', p_true
    print 'True conf matrices'
    for i, mat in enumerate(trueConfMat.values()): 
        print 'Classifier:%s'%name_classifier[i]
        print str_mat(mat)
        print
    #--------------------------------------------------------
    #.         Conflict Graph and Spectral Estimator
    #--------------------------------------------------------
    # init params for spectral estimator and the conflict graph
    params = {}
    unip = np.ones(num_class)/float(num_class);
    # the initial conf matries
    expert_frac = 0.01
    expertConfMat = {j: (expert_frac)*np.eye(num_class)+ 
                     (1-expert_frac)*np.tile(unip, [num_class, 1])
                     for j in range(num_classifier)}
    if truePrior:
        # True prior
        params['Confmat'] = trueConfMat
        params['p_true'] = p_true
    else:
        # expert prior with uniform label porbabilities
        params['Confmat'] = expertConfMat
        params['p_true'] = unip
    #---------display: current parameter error--------------
    est_err = ConfError(trueConfMat, 0, p_true, num_classifier, 
                         params['Confmat'], params['p_true'])
    
    true_avg_acc = [averageAcc(p_true, trueConfMat[i]) for i in range(num_classifier)]
    init_avg_acc = [averageAcc(p_true, params['Confmat'][i]) for i in range(num_classifier)]
    print 'True Avg accuracy: %s'%str_arr(true_avg_acc)
    print 'Initial Avg accuracy: %s'%str_arr([averageAcc(p_true, params['Confmat'][i]) 
                                              for i in range(num_classifier)])
    
    #                Allowed subsets
    
    k_max_size = min(num_classifier-1, k_max_size)
    k_max = [num_classifier]
    for kms in range(k_max_size):
        k_max.append(int(kms+1))
        k_max.append(num_classifier - int(kms+1))
    k_max = list(set(k_max))
    
    allowed_subset = []
    for k in k_max:
        allowed_subset += kbits(int(num_classifier), int(k))
    
    #           Create the conflict graph
    
    disp = 100    # disp parameter for conflict Graph
    G = conflictGraph(allowed_subset = allowed_subset, M = num_classifier, 
                      C = num_class, Ens= ENS, params = params, 
                      V = V, thres = thres, disp = disp)
    
    #          Create the spectral estimator
    
    S = spectralEM(num_classifier, num_class, maxiter = 250, num_init = 50)
    
    #          Compute metrics for Spectral estimation
    
    group = S.group 
    groupConfMat = {i: np.mean([trueConfMat[j] for j in range(num_classifier) if group[i][j]], axis = 0)  
                    for i in range(3)}

    kappa = np.min([np.min([[groupConfMat[j][l,l] - groupConfMat[j][l,c] for c in range(num_class) if c !=l] 
                          for l in range(num_class)]) for j in range(3)])


    barD = np.min([[np.mean([entropy(trueConfMat[i][l,:], trueConfMat[j][l,:]) 
                             for i in range(num_classifier)]) 
                    for c in range(num_class) if c !=l] 
                   for l in range(num_class)])
    
    #Display: metrics for spectral estimation
    print 'Metrics in Spectral Learning| kappa:%.3f, barD:%.3f'%(kappa, barD)
    print
    
    #display: allowed subsets
    print 'Allowed all subsets of size:',k_max

    #              Reset Internal States
    
    G.reset() # resets the internal queues
    S.reset() # resets the internal parameters

    #       Exploration and estimation parameters 
    
    update_num = 25 # lenght of explore_data when S is updated
    updateSpectral = True # no spectral updates (only EM)
    # make the following two 0 to stop exploration
    explore_prob_const = 0.1 # probability with which exploration happens
    init_explore = 200 # Number of initial time slots when explore happens
    #---------------------------------------------------
    
    # Initialization
    output_stream= []
    true_label_stream = []
    queue_evol = []
    error_evol = []
    est_evol = []

    samp_count = 0
    tau = 0 
    explore_data = []
    count_update = 0
    
    # display period
    disp_period = 25
    display = 2 # display flag
    
    print 'Arrival Rate:', arr_rate, 'Accuracy Threshold:', G.thres
    print 'Total samples:', num_tot_samp, 'Total time slots:', time_slots
    #--------------------------------------
    #            Iterations
    #--------------------------------------
    print '----------\nIterations Begin\n-----------' 
    while len(output_stream)< num_tot_samp:
        #----arrival-----
        if tau < time_slots:
            # sample id should begin from 1 while providing newArrival
            G.newArrival(stream_sample[samp_count:samp_count+arrival_arr[tau]]+1)
            samp_count += arrival_arr[tau]
        
        #----------------------------------------------------------
        #                  Exploit or Explore: Scheduling
        #----------------------------------------------------------
        
        #                set explore probability 
        
        # default
        explore_prob = explore_prob_const # explore_prob_const*np.log(i)/i
        # explore for the first 100 rounds
        explore_prob = 1.0 if tau < init_explore else explore_prob
        # explore only if updateON
        explore_flag = (np.random.rand() < explore_prob) and updateON
        
        #                 schedule
        
        try:
            sample_out, schedule, _ = G.schedule(explore = explore_flag)
        except Exception as e:
            print '***Error in Scheduling***Exiting***'
            traceback.print_exc(); break

        #----------------------------------------------------------
        #                  Update Parameters
        #----------------------------------------------------------
        if updateON: # update on each output
        #if explore_flag: # update on exploration instances
            explore_data += [s[2]-1 for s in sample_out]
            
        if (len(explore_data) >= update_num) and updateON:
            
            #         update the spectralEM object
            
            S.update(explore_data);
            
            #          recallibrate params using Spectral
            
            if (tau > init_explore) and updateSpectral:
                S.updateParamsSpectral()
                count_update +=1
                
            #           update params using EM
            
            S.updateParamsEM(explore_data);
            
            #           update G params
            
            new_params = {'Confmat': S.conf_mat, 'p_true': S.ph_est_avg}
            G.updateParams(new_params)
            
            #            compute error 
            
            est_err = ConfError(trueConfMat, S.num_data, p_true, 
                           num_classifier, S.conf_mat, S.ph_est_avg)
            
            #           empty explore_data 
            
            explore_data = []
        #----------------------------------------------------------       
        #                    Results
        #----------------------------------------------------------
        true_label_out = []
        for s in sample_out:
            ind_s, = np.where(stream_sample == int(s[0]-1))
            if stream_sample[ind_s] != int(s[0]-1): 
                print "Error in finding sample location"
                true_label_out += [-1]
            else:
                true_label_out += [stream_true_labels[ind_s]+1]
            
        rem_samples = G.totalCount()
        output_stream += sample_out
        tot_queue = G.totalCount()
        true_label_stream += true_label_out
        classification_error = computeError(output_stream, true_label_stream)

        queue_evol.append(tot_queue) 
        error_evol.append(classification_error)
        est_evol.append(est_err)
        #----------------------------------------------------------       
        #                   Display
        #----------------------------------------------------------
        if not np.mod(tau, disp_period): 
            print '--------------------'
            print 'num iterations:', tau;
            print 'Total samples in queues:', G.totalCount()
            print 'Total samples in explore list:', len(explore_data)
            print 'Number of outputs:', len(output_stream)
            print 'Error fraction:%.3f'%classification_error
            print '-------------------'
        if display <= 1:
           
            G.print_labelSubsets()
            print 
        if display <= 2:
            print 'Iteration:', tau, 'Total Queue Length:', tot_queue
            print 'Schedule:', schedule
            print 'Output'
            for i_s, s in enumerate(sample_out):
                # s: sample_id, label_final, label_arr, label_conf
                print " ".join(['Sample Id:%4d'%s[0],
                                'Labels=> Real:%2d'%(true_label_out[i_s]), 
                                'Final:%2d'%s[1], 'Conf:%.3f'%s[3],
                                'All: %s'%str_arr(s[2], 'int')])
            print
                
        #  increase the time count
        
        tau += 1
    #----------------------outputs-----------
    output_dict = {}
    output_dict['tau'] = tau
    output_dict['time_slots'] = time_slots
    output_dict['arrival_arr'] = arrival_arr
    output_dict['output_stream'] = output_stream
    output_dict['true_label_stream'] = true_label_stream
    output_dict['queue_evol'] = queue_evol
    output_dict['error_evol'] = error_evol
    output_dict['est_evol'] = est_evol
    output_dict['rem_samples'] = rem_samples
    #-------------------------------------------
    return output_dict

In [10]:
def displayResults(output_dict):
    tau = output_dict['tau']
    arrival_arr = output_dict['arrival_arr']
    time_slots = output_dict['time_slots']
    output_stream = output_dict['output_stream'] 
    true_label_stream = output_dict['true_label_stream']
    queue_evol = output_dict['queue_evol']
    error_evol = output_dict['error_evol']
    est_evol = output_dict['est_evol'] 
    rem_samples = output_dict['rem_samples'] 
    #-------------------------------------------------------
    print 'Remaining Samples:', rem_samples
    print 'Len O/p Stream:', len(output_stream)
    error_frac = computeError(output_stream, true_label_stream)
    print '-------\nFinal Output\n--------'
    print 'All samples cleared after no. iterations:',tau
    print 'Accuracy:%.3f'%(1- error_frac)
    print 'Time average of total Queue Length:%.2f'%np.mean(queue_evol)

    if tau <= (time_slots):
        arrival_arr_pad = arrival_arr[:tau]
    else:
        arrival_arr_pad = np.array(list(arrival_arr) + list(np.zeros(tau - time_slots)))

    plt.plot(range(tau), arrival_arr_pad, 'b-', range(tau), queue_evol, 'r--')
    plt.legend(['Arrival', 'Total Queue Length'])
    plt.title('Arrival and Evolution of Queues')
    plt.xlabel('timeslot')
    plt.show()



    plt.plot(range(tau), [1-e for e in error_evol], 'b-')
    plt.title('Evolution of Classification Accuracy')
    plt.xlabel('timeslot')
    plt.ylabel('Accuracy')
    plt.show()

    plt.plot(range(tau), [e[0] for e in est_evol], 'b-', range(tau),[np.mean(e[1]) for e in est_evol] , 'r--')
    plt.legend(['p_true', 'conf mat'])
    plt.title('Evolution of Estimation Error')
    plt.xlabel('timeslot')
    plt.show()

In [15]:
if __name__ =='__main__':
    # Compute the results 
    output_dict = simulator(num_class = 3, num_classifier = 5,  # Ensemble and Data
                  load_ENS = False, fit_ENS = False, 
                  load_params = True, save_params = False, # Ensemble
                  k_max_size = 3, # allowed subset 
                  arr_rate = 1.6, thres = 0.6, V = 0,# arrival and departure thres
                  updateON = False, truePrior = True, # priors
                  time_slots = 200)# simulation length
    # Display the results    
    displayResults(output_dict)

Number of class:3
Number of classifiers:5
Name of classifiers: ['syn-0', 'syn-1', 'syn-2', 'syn-3', 'syn-4']
True label probability: [0.35442305 0.25298436 0.39259259]
True conf matrices
Classifier:syn-0
0.468 0.345 0.187
0.154 0.598 0.248
0.192 0.196 0.612

Classifier:syn-1
0.449 0.283 0.268
0.217 0.650 0.133
0.272 0.300 0.428

Classifier:syn-2
0.639 0.011 0.350
0.376 0.608 0.016
0.366 0.020 0.615

Classifier:syn-3
0.719 0.253 0.028
0.196 0.691 0.113
0.107 0.312 0.581

Classifier:syn-4
0.639 0.013 0.347
0.274 0.695 0.032
0.156 0.240 0.604

---------------------------
*****Param Update*****
---------------------------
Num samples:0
True ph:0.354 0.253 0.393
Est ph:0.354 0.253 0.393
L1 error in class probability:0.000
Sum of L1 error in conf matrices:0.000
---------------------------
True Avg accuracy: 0.557 0.492 0.622 0.658 0.639
Initial Avg accuracy: 0.557 0.492 0.622 0.658 0.639
Metrics in Spectral Learning| kappa:0.199, barD:0.191

Allowed all subsets of size: [1, 2, 3, 4, 5]
Arriv

Iteration: 39 Total Queue Length: 1.0
Schedule: [57. 56. 56. 54. 56.]
Output
Sample Id:  57 Labels=> Real: 1 Final: 3 Conf:0.651 All:  3  0  0  0  0
Sample Id:  54 Labels=> Real: 3 Final: 3 Conf:0.947 All:  0  2  1  3  3

Iteration: 40 Total Queue Length: 0.0
Schedule: [56.  0.  0.  0.  0.]
Output
Sample Id:  56 Labels=> Real: 1 Final: 1 Conf:0.738 All:  1  1  1  2  3

Iteration: 41 Total Queue Length: 1.0
Schedule: [58.  0. 59. 58.  0.]
Output
Sample Id:  58 Labels=> Real: 3 Final: 3 Conf:0.940 All:  3  0  0  3  0

Iteration: 42 Total Queue Length: 1.0
Schedule: [ 0. 59.  0.  0. 59.]
Output

Iteration: 43 Total Queue Length: 2.0
Schedule: [60. 60. 61. 60. 62.]
Output
Sample Id:  60 Labels=> Real: 3 Final: 3 Conf:0.758 All:  3  3  0  2  0
Sample Id:  62 Labels=> Real: 2 Final: 2 Conf:0.640 All:  0  0  0  0  2

Iteration: 44 Total Queue Length: 4.0
Schedule: [63. 63. 64. 64. 64.]
Output

Iteration: 45 Total Queue Length: 5.0
Schedule: [66. 65. 66. 65. 65.]
Output
Sample Id:  66 Labels=>

Iteration: 84 Total Queue Length: 11.0
Schedule: [128. 124. 101. 113. 113.]
Output
Sample Id: 124 Labels=> Real: 3 Final: 3 Conf:0.669 All:  2  3  3  2  0
Sample Id: 101 Labels=> Real: 2 Final: 2 Conf:0.917 All:  0  3  2  2  0

Iteration: 85 Total Queue Length: 11.0
Schedule: [100. 128.   0. 128. 128.]
Output
Sample Id: 128 Labels=> Real: 1 Final: 1 Conf:0.939 All:  2  1  1  1  3

Iteration: 86 Total Queue Length: 12.0
Schedule: [130.   0. 131. 130. 130.]
Output
Sample Id: 131 Labels=> Real: 2 Final: 2 Conf:0.931 All:  0  0  2  0  0

Iteration: 87 Total Queue Length: 12.0
Schedule: [  0.   0. 132.   0. 132.]
Output
Sample Id: 132 Labels=> Real: 1 Final: 3 Conf:0.771 All:  0  0  3  0  3

Iteration: 88 Total Queue Length: 12.0
Schedule: [  0.   0. 133.   0. 133.]
Output
Sample Id: 133 Labels=> Real: 3 Final: 2 Conf:0.638 All:  0  0  1  0  2

Iteration: 89 Total Queue Length: 13.0
Schedule: [135. 134. 136.   0. 136.]
Output
Sample Id: 136 Labels=> Real: 3 Final: 3 Conf:0.771 All:  0  0  3


--------------------
num iterations: 125
Total samples in queues: 3.0
Total samples in explore list: 0
Number of outputs: 182
Error fraction:0.236
-------------------
Iteration: 125 Total Queue Length: 3.0
Schedule: [  0. 177. 177. 177. 185.]
Output
Sample Id: 185 Labels=> Real: 1 Final: 1 Conf:0.634 All:  0  0  0  0  1

Iteration: 126 Total Queue Length: 3.0
Schedule: [186. 186. 186. 186. 186.]
Output
Sample Id: 186 Labels=> Real: 2 Final: 1 Conf:0.827 All:  2  2  1  1  1

Iteration: 127 Total Queue Length: 3.0
Schedule: [188. 188. 188. 187. 187.]
Output
Sample Id: 187 Labels=> Real: 3 Final: 3 Conf:0.733 All:  0  0  0  3  2
Sample Id: 188 Labels=> Real: 1 Final: 1 Conf:0.627 All:  1  2  1  0  0

Iteration: 128 Total Queue Length: 4.0
Schedule: [190. 190. 190. 189. 189.]
Output
Sample Id: 189 Labels=> Real: 3 Final: 3 Conf:0.969 All:  0  0  0  3  3

Iteration: 129 Total Queue Length: 3.0
Schedule: [  0.   0.   0. 190.   0.]
Output
Sample Id: 190 Labels=> Real: 3 Final: 3 Conf:0.926 A

Iteration: 169 Total Queue Length: 17.0
Schedule: [256. 256. 255.   0. 256.]
Output
Sample Id: 256 Labels=> Real: 2 Final: 3 Conf:0.804 All:  3  3  0  0  2

Iteration: 170 Total Queue Length: 18.0
Schedule: [258. 257. 257. 259. 258.]
Output
Sample Id: 258 Labels=> Real: 2 Final: 2 Conf:0.839 All:  2  0  0  0  2
Sample Id: 259 Labels=> Real: 1 Final: 1 Conf:0.735 All:  0  0  0  1  0

Iteration: 171 Total Queue Length: 18.0
Schedule: [260.   0. 260. 262. 261.]
Output
Sample Id: 260 Labels=> Real: 2 Final: 2 Conf:0.970 All:  2  0  2  0  0
Sample Id: 261 Labels=> Real: 1 Final: 1 Conf:0.634 All:  0  0  0  0  1
Sample Id: 262 Labels=> Real: 3 Final: 3 Conf:0.856 All:  0  0  0  3  0

Iteration: 172 Total Queue Length: 18.0
Schedule: [  0. 263.   0.   0. 263.]
Output
Sample Id: 263 Labels=> Real: 3 Final: 2 Conf:0.794 All:  0  2  0  0  2

Iteration: 173 Total Queue Length: 18.0
Schedule: [264.   0. 264. 266. 265.]
Output
Sample Id: 264 Labels=> Real: 3 Final: 3 Conf:0.859 All:  3  0  3  0  0


Iteration: 208 Total Queue Length: 5.0
Schedule: [  0.   0. 213.   0. 213.]
Output
Sample Id: 213 Labels=> Real: 2 Final: 2 Conf:0.991 All:  3  2  2  2  2

Iteration: 209 Total Queue Length: 4.0
Schedule: [  0.   0. 180.   0. 180.]
Output
Sample Id: 180 Labels=> Real: 2 Final: 1 Conf:0.627 All:  2  3  1  2  1

Iteration: 210 Total Queue Length: 3.0
Schedule: [  0.   0.   0.   0. 244.]
Output
Sample Id: 244 Labels=> Real: 2 Final: 2 Conf:0.474 All:  3  2  1  2  1

Iteration: 211 Total Queue Length: 2.0
Schedule: [  0.   0.   0.   0. 233.]
Output
Sample Id: 233 Labels=> Real: 3 Final: 3 Conf:0.716 All:  3  1  1  2  3

Iteration: 212 Total Queue Length: 1.0
Schedule: [  0.   0.   0.   0. 158.]
Output
Sample Id: 158 Labels=> Real: 2 Final: 1 Conf:0.686 All:  1  2  1  2  1

Iteration: 213 Total Queue Length: 0.0
Schedule: [  0.   0.   0.   0. 177.]
Output
Sample Id: 177 Labels=> Real: 1 Final: 3 Conf:0.530 All:  2  3  1  2  3

